#Date le definizioni riconoscere i concetti di cui parlano (content-to-form)

Import


1.   content-to-form.csv
2.   stop words FULL



In [ ]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import semcor
from xml.dom import minidom
import random
from numpy import mean
import pandas as pd
from nltk.corpus.reader.wordnet import Lemma
import csv
from nltk.probability import FreqDist
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
import spacy
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#Funzioni ausiliarie e Algoritmo di Lesk

In [ ]:
lemmatizer = WordNetLemmatizer() 
stop_words_list = []

# Ritorna il POS TAG di una parola
def get_wordnet_pos(word):
    treebank_tag = [tag for (word, tag) in nltk.pos_tag(nltk.word_tokenize(word))][0]
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return ''

# Ritorna una lista con le stop words
def get_stop_words() :
    if len(stop_words_list) == 0 :
        f = open("stop_words_FULL.txt", "r")
        for x in f:
            stop_words_list.append(x)
    
    return stop_words_list

# Da una frase ritorna una lista con le singole parole (lemmi) rimuovendo le parole inutili (stop words..)
def get_list_of_gains_words(sentence) :
    list_words_lemma = []
    aus_list_words = nltk.word_tokenize(sentence)
    stop_words_list = get_stop_words()

    for w in aus_list_words :
        if w.lower() not in stop_words_list :
            pos_tag = get_wordnet_pos(w)
            if pos_tag != '' :
                list_words_lemma.append(lemmatizer.lemmatize(w.lower(), pos_tag))

    return list_words_lemma

# Ritorna la lista delle parole della signature di un synset : gloss + examples
def get_signature_of_synset(synset) :
    # Gloss
    signature = get_list_of_gains_words(synset.definition())
    # Examples
    for ex in synset.examples() :
        list_ex = get_list_of_gains_words(ex)
        signature.extend(list_ex)

    return signature

# Calcola il numero di elementi comuni di due liste
def overlap_lists(list1, list2) :
    common_elements = set(list1) & set(list2)
    return len(common_elements)

# Algoritmo di Lesk
def lesk_algorithm(word, sentence) :
    best_sense = wn.synsets(word)[0]
    max_overlap = 0
    context = get_list_of_gains_words(sentence)
    for sense in wn.synsets(word) :
        #Signature
        signature_sense = get_signature_of_synset(sense)
        if len(signature_sense) < 25 : 
            #Aggiungo le info degli iponomi
            for hypo in sense.hyponyms() :
                signature_sense.extend(get_signature_of_synset(hypo))  
              
        #Overlap
        overlap = overlap_lists(context, signature_sense)

        if overlap > max_overlap :
            max_overlap = overlap
            best_sense = sense

    return best_sense

In [ ]:
#leggo dal csv le definizioni
df = pd.read_csv (r'defs.csv')

def get_definition_for_concept(concept):
    concept_array= df[[concept]].to_numpy()
    concept_list=[]
    for arr in concept_array:
      concept_list.append(str(arr[0]))
    return concept_list
    

In [ ]:
# Data la lista delle parole, la ritorna ordinata in base alla frequenza
def get_ordered_list_words(definizioni):
    common_words = []
    res = [' '.join(ele) for ele in definizioni]
    def_text=""
    for x in res:
      def_text += " "+x
    #usiamo FreqDist per calcolare la frequenza dei token
    fdist1 = FreqDist(word.lower() for word in word_tokenize(def_text)) # freqDist da una distribuzione di frequenza 
    for tupla in fdist1.most_common():
      lista= list (tupla)
      common_words.append(lista[0])
    return common_words

# Dato un synset, scorre in ampiezza gli iponimi per vedere se ne trova uno con score migliore, se lo trovo richiamo
#il metodo ricorsivamente per vedere se scendendo miglioro ancora lo score
def rec_hypons(sense, value, list_words) :
    best_sense = ""
    best_val = 0
    #per ogni iponimo tra gli iponimi di sense
    for hypo in sense.hyponyms() :
        #ricalcolo intersezione quindi gli elementi comuni tra le due liste passate come parametri
        intersection = overlap_lists(list_words, get_signature_of_synset(hypo))
        if best_val <= intersection :
            best_val = intersection
            best_sense = hypo

    # Se ho trovato un valore migliore del genus provo ancora a scendere di livello
    if best_val >= value :
        sense,value = rec_hypons(best_sense, best_val, list_words)
    
    return sense, value




courage_definitions=get_definition_for_concept("Courage") #astratto generico
courage_definitions = [x for x in courage_definitions if str(x) != 'nan'] #rimuovo i nan dalla lista

paper_definitions=get_definition_for_concept("Paper")
paper_definitions= [x for x in paper_definitions if str(x) != 'nan']#rimuovo i nan dalla lista 

apprehension_definitions=get_definition_for_concept("Apprehension")
apprehension_definitions= [x for x in apprehension_definitions if str(x) != 'nan']#rimuovo i nan dalla lista 

sharpener_definitions=get_definition_for_concept("Sharpener")
sharpener_definitions= [x for x in sharpener_definitions if str(x) != 'nan']#rimuovo i nan dalla lista 




for i in range(len(courage_definitions)):
    courage_definitions[i] = get_list_of_gains_words(courage_definitions[i])
for i in range(len(paper_definitions)):
    paper_definitions[i] = get_list_of_gains_words(paper_definitions[i])
for i in range(len(sharpener_definitions)):
    sharpener_definitions[i] = get_list_of_gains_words(sharpener_definitions[i])
for i in range(len(apprehension_definitions)):
    apprehension_definitions[i] = get_list_of_gains_words(apprehension_definitions[i])
list_concepts=[courage_definitions,paper_definitions,apprehension_definitions,sharpener_definitions]


In [ ]:
#Lista concetti - preprocessing : stop word - lemmatization
list_def_concepts = list_concepts


# Per ogni concetto da trovare
for c in list_def_concepts :
  # Si prendono tutti i termini a partire dalle definizioni scritte per il concetto, ordinati per frequenza
  concept_words = get_ordered_list_words(c)

  best_sense = ""
  best_val = 0
  backtracking_sense = ""

  #Accodo lista parole per calcolare bag of words
  phrase_words = ""
  for w in concept_words :
    phrase_words += w + " "

  i = 0
  num_nome = 0
  # Scorro il primo nome della lista delle parole del concetto, per cercare il genus
  #Usiamo 1 per num_nome perchè non abbiamo incrementi di accuratezza che giutificano un incremento della complessità
  while i < len(concept_words) and num_nome < 1:
    if (get_wordnet_pos(concept_words[i]) == wn.NOUN ): #controllo postag per prendere solo nomi
      try:
        #Calcolo tramite lesk il synset
        syn = lesk_algorithm(concept_words[i], phrase_words)
        num_nome +=1

        #Calcolo tra i nomi analizzati dalla lista delle parole, il synset con valore migliore
        # ossia conto il numero di parole comuni tra le parole che definiscono il concetto
        # e la signature del synset, che metto in intersection (più alto è meglio)
        intersection = overlap_lists(concept_words, get_signature_of_synset(syn)) 
        
        if intersection < best_val or best_val == 0: # salvo genus per backtracking
          backtracking_sense = syn
          #print("backtracking_sense " + str(backtracking_sense) +" "+ str(type(backtracking_sense)) )
          #print("intersection nacktracking " + str(intersection))

        # Se il synset ha valore bag-of-words migliore, lo uso come genus
        if best_val <= intersection :
          best_val = overlap_lists(concept_words, get_signature_of_synset(syn))
          best_sense = syn
          #print("Best VAL " + str(best_val) +" "+ str(type(best_val)) )
          #print("Best SENSE " + str(best_sense) +" "+ str(type(best_sense)))                            
      except : pass
    i+=1

  # Metodo ricorsivo per cercare tra gli iponimi del genus un synset migliore
  best_ipo, score_best_ipo = rec_hypons(best_sense, best_val, concept_words)

  # Se l'iponimo trovato non migliora il valore del genus
  if score_best_ipo < best_val: # Backtracking --> provo l'altro nome scartato come genus
    best_ipo_bkt, score_best_ipo_bkt = rec_hypons(backtracking_sense, best_val, concept_words)
    if score_best_ipo_bkt > score_best_ipo:
      best_ipo = best_ipo_bkt

  # Visualizzazione miglior synset che descrive il concetto delle definizioni                    
  print(best_ipo)
  print(best_ipo.definition())
  print("\n")

Synset('ability.n.01')
the quality of being able to perform; a quality that permits or facilitates achievement or accomplishment


Synset('writing_paper.n.01')
paper material made into thin sheets that are sized to take ink; used for writing correspondence and manuscripts


Synset('feel.v.01')
undergo an emotional sensation or be in a particular state of mind


Synset('slate_pencil.n.01')
a pencil of soft slate (or soapstone) used for writing on a slate




#Output

Concetti: courage, paper, apprehension, sharpener

```
Synset('ability.n.01')
the quality of being able to perform; a quality that permits or facilitates achievement or accomplishment


Synset('writing_paper.n.01')
paper material made into thin sheets that are sized to take ink; used for writing correspondence and manuscripts


Synset('feel.v.01')
undergo an emotional sensation or be in a particular state of mind


Synset('slate_pencil.n.01')
a pencil of soft slate (or soapstone) used for writing on a slate
```

